In [ ]:
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')

In [ ]:
from imports import *
from encoders import TargetEncoder

In [ ]:
def get_company_okved_code(dataset_path: str, company_okved_dict_path: str):
    """
    Строит словарь id компании - код ОКВЭД
    Args:
        dataset_path (str): путь к каталогу с pickle файлами с информацией о ГСК
        company_okved_dict_path (str): путь для сохранения словаря с ОКВЭД компании
        
    Returns:
        dict: словарь id компании - код ОКВЭД 
    """
    if isfile(company_okved_dict_path):
        return pickle.load(open(company_okved_dict_path, 'rb'))
    
    company_okved = {}
    with Path(dataset_path) as path:
        for file in tqdm(path.iterdir()):
            gc = pickle.load(open(file, 'rb'))
            for company in gc['nodes']:
                company_okved[company['id']] = company['okved_code']
    
    with open(company_okved_dict_path, 'wb') as fp:
        pickle.dump(company_okved, fp)
    return company_okved

In [ ]:
def generate_financial_features(fin_features: pd.DataFrame) -> pd.DataFrame:
    """
    Фильтрует данные о финансовых показателях компании, оставляя только записи, относящиеся к компаниям,
    для которых есть информация за несколько лет и известен код ОКВЭД
    Args:
        fin_features (pd.DataFrame): Таблица с финансовыми показателями компаний за несколько лет
    Returns:
        pd.DataFrame: Таблица с финансовыми показателями компаний, в которой нет столбцов с нулями
                      и для каждой компании есть информация хотя бы за 2 года, отсортированная
                      по ID компании и году
    """
    profit_col = 'p24004'
    fin_statement_cols = ['arango_id', 'balance_year', profit_col]

    for col in [1100, 1200, 1210, 1230, 1250, 1300, 1500, 1520, 1600, 1700, 2340, 2350]:
        fin_statement_cols.extend([f'p{col}3', f'p{col}4'])

    fin_features = fin_features[fin_statement_cols].copy()
    fin_features.drop_duplicates(subset=['arango_id', 'balance_year'], inplace=True)
    fin_features = fin_features[(fin_features[fin_statement_cols] == 0).sum(axis=1) == 0]

    company_years = fin_features['arango_id'].value_counts()
    have_multiple_years = company_years[company_years > 1].index
    fin_features = fin_features[(fin_features['arango_id'].isin(have_multiple_years)) &
                                (fin_features['arango_id'].isin(company_okved))]

    fin_features.sort_values(['arango_id', 'balance_year'], inplace=True)
    fin_features.reset_index(drop=True, inplace=True)
    fin_features.insert(2, 'okved_code', fin_features['arango_id'].map(company_okved))
    fin_features[profit_col] = (fin_features[profit_col] >= 0).astype(int)
    fin_features.rename(columns={profit_col: 'has_profit'}, inplace=True)
    fin_features.pop('balance_year')
    return fin_features


def to_ptc_changes(df: pd.DataFrame) -> pd.DataFrame:
    """
    Считает процентные изменения финансовых показателей компаний относительно предыдущего года

    Args:
        df (pd.DataFrame): Таблица с финансовыми показателями компаний, отсортированная
                           по ID компании и году
    Returns:
        pd.DataFrame: Таблица с процентными изменениями финансовых показателей
    """

    df = df.copy()
    for col in df.columns[3:]:
        shifted_vals = df[col].shift().where(df['arango_id'] == df['arango_id'].shift())
        df[col] = (df[col] - shifted_vals) / shifted_vals
    return df.dropna()

In [ ]:
def create_dataset0(X_train: np.array, X_test: np.array, y_train: np.array, y_test: np.array) -> tuple:
    """
    Создает датасет без информации о коде ОКВЭД
    """
    X_train.pop("okved_code")
    X_test.pop('okved_code')
    dset = X_train, X_test, y_train, y_test
    return dset


In [ ]:
def create_dataset1(X_train: np.array, X_test: np.array, y_train: np.array, y_test: np.array)-> tuple:
    """
    Создает датасет с кодами ОКВЭД, закодированными при помощи Target Encoding
    """
    okved_encoder = TargetEncoder().fit(X_train['okved_code'], y_train)
    X_train['okved_code'] = okved_encoder.transform(X_train['okved_code'])
    X_test['okved_code'] = okved_encoder.transform(X_test['okved_code'])
    dset = X_train, X_test, y_train, y_test
    return dset


In [ ]:
def create_dataset2(X_train: np.array, X_test: np.array,
                    y_train: np.array, y_test: np.array,
                    okved_embeddings_dict: dict) -> tuple:
    """
    Создает датасет с кодами ОКВЭД, закодированными при умных эмбеддингов
    """
    emb_dim = len(okved_embeddings_dict[list(okved_embeddings_dict.keys())[0]])
    new_cols = [f'okved_code_{i}' for i in range(emb_dim)]
    X_train[new_cols] = X_train['okved_code'].map(lambda x: okved_embeddings_dict.get(x, np.zeros(emb_dim))).tolist()
    X_test[new_cols] = X_test['okved_code'].map(lambda x: okved_embeddings_dict.get(x, np.zeros(emb_dim))).tolist()
    X_train.pop('okved_code')
    X_test.pop("okved_code")
    dset = X_train, X_test, y_train, y_test
    return dset

In [ ]:
def train_model(dset: tuple, n_epochs: int, print_each: int, lr: float = 0.005) -> tuple:
    """
    Обучает однослойную НС на данном датасете
    Args:
        dset (tuple): кортеж из 4 массивов: X_train, X_test, y_train, y_test
        n_epochs (int): кол-во эпох для обучения
        print_each (int): шаг для вывода текущей информации во время обучения
        lr (float, optional): скорость обучения
    Returns:
        tuple: обученная модель и метрики качества
    """
    X_train, X_test, y_train, y_test = [torch.from_numpy(df.values) for df in dset]
    X_train = X_train.float()
    X_test = X_test.float()
    y_train = y_train.long().flatten()
    y_test = y_test.long().flatten()

    n_feats = X_train.shape[1]
    out_classes = 2
    model = nn.Sequential(nn.Linear(n_feats, 2))
    criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor(compute_class_weight('balanced',
                                                                                  classes=np.array([0, 1]),
                                                                                  y=y_train.numpy())))
    metrics = {
        'train_auc': [],
        'test_auc': []
    }

    optimizer = optim.Adam(model.parameters(), lr=lr)
    for epoch in range(n_epochs):
        model.train()
        logits = model(X_train)
        loss = criterion(logits, y_train)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        train_logits = logits.detach()
        train_auc = roc_auc_score(y_train, train_logits[:, 1])

        logits_test = model(X_test).detach()
        test_auc = roc_auc_score(y_test, logits_test[:, 1])

        if epoch % print_each == 0 or epoch == n_epochs - 1:
            print(f'{epoch=:05d} | loss={loss.item():.4f} | train_auc={train_auc:.4f} ')
        metrics['train_auc'].append(train_auc)
        metrics['test_auc'].append(test_auc)

    return model, metrics

In [ ]:
# загружаем конфигурационный файл
CONFIG = yaml.safe_load(open('CONFIG.yaml', encoding='utf8'))

# получаем код ОКВЭД для каждой компании из датасета
company_okved = get_company_okved_code(CONFIG['paths']['gc_augmented_save'], CONFIG['paths']['company_okved_dict'])

with open(CONFIG['paths']['okved_embeddings'], 'rb') as fp:
    okved_embeddings_dict = pickle.load(fp)

# создаем таблицу с финансовыми показателями
if isfile(CONFIG['paths']['profits_dataset']):
    fin_features_preproc = pd.read_csv(CONFIG['paths']['profits_dataset'])
else:
    fin_features = pd.read_csv(CONFIG['paths']['fin_data'])
    fin_features_preproc = generate_financial_features(fin_features.copy())
    fin_features_preproc = to_ptc_changes(fin_features_preproc)
    fin_features_preproc.to_csv(CONFIG['paths']['profits_dataset'], index=False)

display(fin_features_preproc.head(2))

# создаем несколько версий датасета
X = fin__features_preproc.drop(columns=['has_profit', 'arango_id'])
y = fin_features_preproc['has_profit']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
dset0 = create_dataset0(X_train.copy(), X_test.copy(), y_train.copy(), y_test.copy())
dset1 = create_dataset1(X_train.copy(), X_test.copy(), y_train.copy(), y_test.copy())
dset2 = create_dataset2(X_train.copy(), X_test.copy(), y_train.copy(), y_test.copy(), okved_embeddings_dict)


In [ ]:
# обучаем 1 модель
model0, metrics0 = train_model(dset0, n_epochs=500, print_each=50, lr=0.05)

In [ ]:
# обучаем 2 модель
model1, metrics1 = train_model(dset1, n_epochs=500, print_each=50, lr=0.05)

In [ ]:
# обучаем 3 модель
model2, metrics2 = train_model(dset2, n_epochs=500, print_each=50, lr=0.05)

In [ ]:
def plot_roc_auc(n_epochs: int = None, *model_metrics):
    """
    Рисует график значений ROC AUC за первые n_epoca эпох
    """
    descs = ['модель без ОКВЭД', 'модель с закодированным ОКВЭД', 'модель с умными эмбеддингами ОКВЭД']
    metric_name = 'ROC AUC'
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))
    for dset_idx, metrics in enumerate(model_metrics):
        ax.plot(metrics[f'test_auc'][:n_epochs], label=f'{desc[dest_idx]}')
        ax.set_title(f'{metric_name} на тестовом множестве')
        ax.set_xlabel('Epoch')
        ax.set_ylabel(f'{metric_name}')
    
    fig.legend(bbox_to_anchor=(0.9, 1.1))


In [ ]:
plot_roc_auc(50, metrics 0, metrics1, metrics2)